[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jzygierewicz/SYNCCIN_2025_summer_school/blob/main/3_channels_MVAR.ipynb)

# SYNCCIN 2025 Summer School - MVAR Connectivity Analysis

# Comparison of Multivariate vs. Bivariate Approach to Connectivity Estimation

This notebook demonstrates advanced connectivity analysis using Multivariate Autoregressive (MVAR) modeling and Directed Transfer Function (DTF) for EEG data analysis. The comparison includes:

- **Bivariate vs. Multivariate MVAR modeling**
- **Spectral analysis and cross-spectral estimation**  
- **Directed Transfer Function (DTF) computation**
- **Interactive visualizations of connectivity patterns**
- **Network graph representations**

## Key Learning Objectives:
1. Understand the differences between bivariate and multivariate approaches
2. Learn MVAR model order selection using information criteria
3. Compute and interpret DTF for connectivity analysis
4. Visualize connectivity patterns using interactive plots and network graphs

## Setup and Installation

First, let's install the required packages for Google Colab:

In [ ]:
# Install required packages for Google Colab
!pip install plotly networkx scipy joblib ipywidgets

# Enable plotly for Colab
import plotly.io as pio
pio.renderers.default = "colab"

## Import Required Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import joblib
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Note: Custom modules (mtmvar and utils) will be imported after downloading
# Run the download section first to get these modules from the GitHub repository

## Download Required Files

Let's download all the required files directly from the GitHub repository:
- Data files (`.joblib` format)
- Python modules (`mtmvar.py` and `utils.py`)

In [ ]:
import urllib.request
import os

# GitHub repository information
github_user = "jzygierewicz"
repo_name = "SYNCCIN_2025_summer_school"
branch = "main"

# Files to download
files_to_download = [
    "simulated_3_channels.joblib",
    "mtmvar.py",
    "utils.py"
]

print("Downloading required files from GitHub repository...")

for filename in files_to_download:
    url = f"https://raw.githubusercontent.com/{github_user}/{repo_name}/{branch}/{filename}"
    
    try:
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
        
        # Check if file was downloaded successfully
        if os.path.exists(filename):
            file_size = os.path.getsize(filename)
            print(f"✓ {filename} downloaded successfully ({file_size:,} bytes)")
        else:
            print(f"✗ Failed to download {filename}")
            
    except Exception as e:
        print(f"✗ Error downloading {filename}: {str(e)}")
        print(f"  You may need to upload this file manually to your Colab session")

print("\nDownload process completed!")
print("\nAvailable files in current directory:")
for f in os.listdir("."):
    if f.endswith((".joblib", ".pkl", ".py")):
        size = os.path.getsize(f)
        print(f"  - {f} ({size:,} bytes)")

# Verify Python modules can be imported
print("\nTesting Python module imports...")
try:
    from mtmvar import mvar_criterion, AR_coeff, mvar_H, mvar_plot
    print("✓ mtmvar module imported successfully")
except ImportError as e:
    print(f"✗ Error importing mtmvar: {e}")

try:
    from utils import describe_dict
    print("✓ utils module imported successfully")
except ImportError as e:
    print(f"✗ Error importing utils: {e}")

print("\nSetup completed! You can now run the analysis.")

## Import Custom Modules

Now that we've downloaded the custom modules, let's import the required functions:

In [ ]:
# Import custom modules that were downloaded from the repository
try:
    from mtmvar import mvar_criterion, AR_coeff, mvar_H, mvar_plot, graph_plot
    print("✓ Successfully imported mtmvar functions.")
except ImportError as e:
    print(f"✗ Error importing mtmvar module: {e}")
    print("Make sure you ran the download section first!")

try:
    from utils import describe_dict
    print("✓ Successfully imported utils functions.")
except ImportError as e:
    print(f"✗ Error importing utils module: {e}")
    print("Make sure you ran the download section first!")

print("\nAll modules ready for MVAR connectivity analysis!")

## Load and Explore Data

Let's load the simulated 3-channel EEG data and explore its characteristics:

In [ ]:
# Load simulation data from simulated_3_channels.joblib
# This file was downloaded automatically from the GitHub repository
data = joblib.load('simulated_3_channels.joblib')
describe_dict(data)

sim1 = data['EEG']
Fs = data['Fs']
N_chan, N_samp = sim1.shape
sim_chan_names = ['0', '1', '2']
print(f"\n=== DATA CHARACTERISTICS ===")
print(f"Number of channels: {N_chan}")
print(f"Number of samples: {N_samp}")
print(f"Sampling frequency: {Fs} Hz")
print(f"Data duration: {N_samp/Fs:.2f} seconds")
print(f"Data shape: {sim1.shape}")

## Visualize the EEG Signals

Let's plot the 3-channel EEG signals using interactive Plotly visualizations:

In [ ]:
# Plot the signals using Plotly for interactive zooming
time_samples = np.arange(N_samp)
time_seconds = time_samples / Fs

fig = make_subplots(
    rows=N_chan, cols=1,
    shared_xaxes=True,
    subplot_titles=[f'Channel {i}' for i in range(N_chan)],
    vertical_spacing=0.05
)

colors = px.colors.qualitative.Set1[:N_chan]

for i in range(N_chan):
    fig.add_trace(
        go.Scatter(
            x=time_seconds,
            y=sim1[i, :],
            mode='lines',
            name=f'Channel {i}',
            line=dict(color=colors[i], width=1),
            showlegend=False
        ),
        row=i+1, col=1
    )

fig.update_layout(
    height=150*N_chan + 100,
    title_text="3-Channel EEG Signals",
    font=dict(size=12)
)

fig.update_xaxes(title_text="Time [seconds]", row=N_chan, col=1)
fig.update_yaxes(title_text="Amplitude [µV]")

fig.show()

# MVAR Model Theory

## Multivariate Autoregressive (MVAR) Model

The MVAR model describes the relationship between channels in a multivariate time series:

$$\mathbf{x}(t) = \sum_{k=1}^{p} \mathbf{A}_k \mathbf{x}(t-k) + \mathbf{e}(t)$$

Where:
- $\mathbf{x}(t)$ is the multichannel signal vector at time $t$
- $\mathbf{A}_k$ are the autoregressive coefficient matrices 
- $p$ is the model order
- $\mathbf{e}(t)$ is the noise vector

## Model Order Selection

We use information criterion AIC to determine the optimal model order $p$.

## Directed Transfer Function (DTF)

DTF quantifies the directional flow of information between channels:

$$DTF_{ij}(f) = |H_{ij}(f)|^2$$

Where $H_{ij}(f)$ is the transfer function from channel $j$ to channel $i$.

# Model Estimation

We will model the signal using a multivariate autoregressive (MVAR) model and compare bivariate vs. multivariate approaches.

Let's set up the analysis parameters:

In [ ]:
# Analysis parameters
max_p = 15          # Maximum model order to test
crit_type = 'AIC'   # Information criterion (AIC or BIC)
f = np.arange(0, 64, 0.1)  # Frequency range for analysis
N_f = f.shape[0]    # Number of frequency points

# SPECTRAL ANALYSIS

First, let us examine the estimation of spectra and cross-spectra using the bivariate and multivariate approaches.

## Bivariate Approach

In the bivariate approach, we estimate the model order separately for each pair of channels. This means we analyze each channel pair independently.

In [ ]:
print("=== BIVARIATE SPECTRAL ANALYSIS ===")
print("Analyzing each channel pair separately...")

# Initialize the bivariate spectrum
S_bivariate = np.zeros((N_chan, N_chan, N_f), dtype=np.complex128)

# Process each channel pair
for ch1 in range(N_chan):
    for ch2 in range(ch1+1, N_chan):
        print(f"\nProcessing channel pair: {ch1} ↔ {ch2}")
        
        # Create bivariate signal
        x = np.vstack((sim1[ch1, :], sim1[ch2, :]))
        
        # Estimate optimal model order for this pair
        crit, p_range, p_opt = mvar_criterion(x, max_p, crit_type, False)
        print(f"  Optimal model order: p = {p_opt}")
        
        # Estimate AR coefficients and compute transfer function
        Ar, V = AR_coeff(x, p_opt)
        H, _ = mvar_H(Ar, f, Fs)
        
        # Compute spectrum for this channel pair
        S_2chan = np.zeros((2, 2, N_f), dtype=np.complex128)
        for fi in range(N_f):
            S_2chan[:, :, fi] = H[:, :, fi].dot(V.dot(H[:, :, fi].T))
        
        # Store results in the global bivariate spectrum matrix
        S_bivariate[ch1, ch1, :] = S_2chan[0, 0, :]
        S_bivariate[ch2, ch2, :] = S_2chan[1, 1, :]
        S_bivariate[ch1, ch2, :] = S_2chan[0, 1, :]
        S_bivariate[ch2, ch1, :] = S_2chan[1, 0, :]

print("\n✓ Bivariate spectral analysis completed!")

## Multivariate Approach

In the multivariate approach, we estimate the model order for all channels simultaneously. This provides a more comprehensive view of the system dynamics.

In [ ]:
print("=== MULTIVARIATE SPECTRAL ANALYSIS ===")
print("Analyzing all channels simultaneously...")

# Use all channels together
x = sim1

# Estimate optimal model order for all channels
crit, p_range, p_opt = mvar_criterion(x, max_p, crit_type, True)
print(f"Optimal model order for all channels: p = {p_opt}")

# Estimate AR coefficients and compute transfer function
Ar, V = AR_coeff(x, p_opt)
H, _ = mvar_H(Ar, f, Fs)

# Compute multivariate spectrum
S_multivariate = np.zeros((N_chan, N_chan, N_f), dtype=np.complex128)
for fi in range(N_f):
    S_multivariate[:, :, fi] = H[:, :, fi].dot(V.dot(H[:, :, fi].T))

print("✓ Multivariate spectral analysis completed!")

# Display model order comparison
print(f"\n=== MODEL ORDER COMPARISON ===")
print("Note: The multivariate approach typically results in more parsimonious models")
print("while the bivariate approach may overfit with higher model orders.")

## Spectral Visualization

Let's create  plots to compare the bivariate and multivariate spectral estimates:

In [ ]:
mvar_plot(S_bivariate, S_bivariate,   f, 'From ', 'To ',  sim_chan_names, 'Bivariate DTF' ,'sqrt')
mvar_plot(S_multivariate, S_multivariate,   f, 'From ', 'To ', sim_chan_names, 'Multivariate DTF' ,'sqrt')
plt.show()

## Spectral Analysis Results

**Key Observations:**

1. **Diagonal Elements (Auto-spectra)**: Show the power spectral density of each individual channel
2. **Off-diagonal Elements (Cross-spectra)**: Reveal the spectral similarities between channel pairs
3. **Alpha Peak**: Look for the characteristic alpha rhythm peak around 8-12 Hz
4. **Approach Differences**: Compare the spectral profiles between bivariate and multivariate methods

**Expected Differences:**
- Bivariate approach may show **narrower spectral peaks** due to higher model orders
- Multivariate approach provides a **more integrated view** of the system dynamics

# DIRECTED TRANSFER FUNCTION (DTF) ANALYSIS

Now let's compute the Directed Transfer Function to analyze directional connectivity patterns.

## Bivariate DTF Computation

In [ ]:
print("=== BIVARIATE DTF ANALYSIS ===")
print("Computing DTF for each channel pair separately...")

# Initialize the bivariate DTF
DTF_bivariate = np.zeros((N_chan, N_chan, N_f), dtype=np.complex128)

# Process each channel pair for DTF
for ch1 in range(N_chan):
    for ch2 in range(ch1+1, N_chan):
        print(f"Computing DTF for channel pair: {ch1} ↔ {ch2}")
        
        # Create bivariate signal
        x = np.vstack((sim1[ch1, :], sim1[ch2, :]))
        
        # Estimate model parameters
        _, _, p_opt = mvar_criterion(x, max_p, crit_type, False)
        Ar, _ = AR_coeff(x, p_opt)
        H, _ = mvar_H(Ar, f, Fs)
        
        # Compute DTF (squared magnitude of transfer function)
        DTF_2chan = np.abs(H)**2
        
        # Store in the global DTF matrix
        DTF_bivariate[ch1, ch2, :] = DTF_2chan[0, 1, :]  # ch1 → ch2
        DTF_bivariate[ch2, ch1, :] = DTF_2chan[1, 0, :]  # ch2 → ch1

print("✓ Bivariate DTF computation completed!")

## Multivariate DTF Computation

In [ ]:
print("=== MULTIVARIATE DTF ANALYSIS ===")
print("Computing DTF for all channels simultaneously...")

# Estimate model parameters for all channels
_, _, p_opt = mvar_criterion(sim1, max_p, crit_type, False)
print(f"Using model order: p = {p_opt}")

Ar, _ = AR_coeff(sim1, p_opt)
H, _ = mvar_H(Ar, f, Fs)

# Compute multivariate DTF
DTF_multivariate = np.abs(H)**2

print("✓ Multivariate DTF computation completed!")

print(f"\n=== DTF ANALYSIS SUMMARY ===")
print(f"DTF matrices computed with {N_f} frequency points")
print(f"Frequency range: {f[0]:.1f} - {f[-1]:.1f} Hz")
print("DTF values represent directional information flow between channels")

## DTF Visualization

Let's create visualizations of the DTF results:

In [ ]:
mvar_plot(S_bivariate, DTF_bivariate,   f, 'From ', 'To ',  sim_chan_names, 'Bivariate DTF' ,'sqrt')
mvar_plot(S_multivariate, DTF_multivariate,   f, 'From ', 'To ', sim_chan_names, 'Multivariate DTF' ,'sqrt')
plt.show()

## Network Visualization

Create the network plots:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
f_range = (8, 12)  # frequency range for alpha band
G_bivariate = graph_plot(DTF_bivariate, axs[0], f, f_range, sim_chan_names, 'Bivariate DTF')
G_multivariate = graph_plot(DTF_multivariate, axs[1], f, f_range, sim_chan_names, 'Multivariate DTF')
plt.tight_layout()
plt.show()

# Key Findings and Interpretation

## Spectral Analysis Comparison
- **Bivariate approach**: Shows higher model orders and potentially narrower spectral peaks
- **Multivariate approach**: More parsimonious models with broader spectral features
- **Cross-spectral patterns**: Differences in coupling estimates between approaches

## DTF Connectivity Analysis
- **Bivariate DTF**: May show spurious connections due to indirect pathways
- **Multivariate DTF**: Better at revealing true direct connections
- **Directional patterns**: Compare the flow patterns between the two approaches

## Network Topology
- **Edge thickness**: Proportional to connection strength
- **Network density**: Compare the number and strength of connections

## Detailed Conclusions

### When estimating spectra and cross-spectra:
1. **Similar overall patterns** between bivariate and multivariate approaches
2. **Higher model orders** in bivariate approach lead to narrower spectral peaks
3. **Computational complexity** is higher for bivariate analysis, especially with many channels

### For DTF connectivity analysis:
1. **Spurious connections** are more common in bivariate DTF estimates
2. **Multivariate DTF** provides more accurate representation of direct connections
3. **Indirect effects** are better controlled in the multivariate approach

### Practical recommendations:
- **Use multivariate approach** when possible for more accurate connectivity estimates
- **Bivariate approach** may be necessary for very high-dimensional data due to computational constraints
- **Model order selection** is important for both approaches
- **Frequency-specific analysis** reveals rhtyhms engaged in the connectivity

## Further Reading and Advanced Topics

### Extended Analysis Methods:
- **Partial Directed Coherence (PDC)**: Alternative to DTF with different normalization
- **Granger Causality**: Statistical approach to directional connectivity
- **Time-varying connectivity**: Analysis of dynamic changes in connectivity
- **Statistical significance testing**: Surrogate data methods for connectivity validation

### Technical Considerations:
- **Stationarity requirements**: MVAR models assume stationary signals
- **Model order selection**: Different criteria (AIC, SC, HQ) may give different results
- **Frequency resolution**: Trade-off between frequency and time resolution
- **Multiple comparison correction**: Important when analyzing many connections


---

**This notebook demonstrates the fundamental differences between bivariate and multivariate approaches to connectivity estimation, providing a foundation for more advanced connectivity analysis methods.**